In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import momepy as mm

In [2]:

region_id = 5883
val_path = '../data/freiburg_morphotopes.pq'

In [3]:

etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/tessellations_chars_{region_id}.parquet')
tess = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/tessellations_chars_{region_id}.parquet')
buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{region_id}.parquet')
streets = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/streets_chars_{region_id}.parquet')
nodes = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/nodes_chars_{region_id}.parquet')
primary = pd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/primary_chars_{region_id}.parquet')

In [4]:
### for freiburg
from core.cluster_validation import generate_detailed_clusters
from core.utils import used_keys

morphotopes = generate_detailed_clusters(etcs, path = val_path)
morphotope_etcs = primary.loc[morphotopes.index]
morphotope_etcs['mlabel'] = morphotopes.values

In [5]:
# morphotope_etcs['types'] = morphotope_etcs['mlabel']

morphotope_etcs['types'] = morphotope_etcs['mlabel'].str.split('_').str[-1]

In [6]:
# morphotope_etcs['types'] = morphotope_etcs['mlabel']
morphotopes = generate_detailed_clusters(etcs, path = val_path)
morphotope_etcs = buildings.loc[morphotopes.index]
morphotope_etcs['mlabel'] = morphotopes.values
morphotope_etcs['types'] = morphotope_etcs['mlabel'].str.split('_').str[-1]

In [1]:
# morphotope_etcs[morphotope_etcs.index >= 0].explore(column='types', prefer_canvas=True, tiles='cartodbpositron')

In [1]:
res = morphotope_etcs.iloc[:, :-2].groupby(morphotope_etcs.mlabel).mean()

res.style.background_gradient(axis=0, cmap="BuGn")

NameError: name 'morphotope_etcs' is not defined

In [25]:
used_keys['stcSAl']

'street alignment of ETC'

In [26]:
## area of enclosure 
## area of perimeter
# 'connected buildings facade ratio'
# 'connected buildings square compactness'
# 'street alignment of ETC'
# perimeter stuff

In [28]:
res = morphotope_etcs.iloc[:, :-2].groupby(morphotope_etcs.types).std().loc[['blocks', 'commercial', 'industry', 'wiehre', 'centre']]
res.style.background_gradient(axis=0, cmap="BuGn")

,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,ssbCCM,ssbCCD,stbOri,mtbSWR,libNCo,ldbPWL,ltcBuA,mtbAli,mtbNDi,ltbIBD,stbCeA,stbSAl,sdsLen,sssLin,ldsMSL,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,sddAre,midRea,midAre,stcOri,sdcLAL,sdcAre,sscCCo,sscERI,mtcWNe,mdcAre,ltcWRB,sicCAR,stcSAl,ldkAre,ldkPer,lskCCo,lskERI,lskCWA,ltkOri,ltkWNB,likWBB,sdsAre,likWCe,mibCou,mibAre,mibLen,mibElo,mibERI,mibCCo,mibLAL,mibFR,mibSCo,micBAD,licBAD,misBAD,midBAD
types,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
blocks,680.057802,71.825222,24.777501,0.136754,9.847066,7.139256,0.111229,0.227105,7.552832,3.544243,16.704621,0.269945,0.408484,298.066899,0.212206,4.827182,15.480946,12.840880,10.338182,13.004386,147.443112,0.101223,44.164155,132.719620,222396.776802,0.124603,8.458037,0.203162,1.843434,1.028118,0.062837,0.109659,0.108831,0.124904,0.000007,0.002813,0.002796,293.065549,0.045105,82.232152,17915.329237,26.037403,43473.159291,15.505609,44.703087,2308.851148,0.137935,0.093876,0.043304,13797.785952,0.000050,0.224558,11.720900,274877.320994,2944.534600,0.121284,0.262246,696.944343,12.369079,0.004589,0.124030,23770.831324,0.001075,11.016558,5981.358352,309.172217,0.246744,0.198835,0.144303,74.085770,4.603906,4.603906,884.575370,651.903159,520.174858,493.378569
commercial,960.600683,69.705440,35.550542,0.122512,4.991043,7.127307,0.088539,0.217513,9.114532,2.662390,10.659070,0.232075,0.552061,201.413207,0.201442,3.921165,14.467002,11.592818,8.409935,8.315829,95.567428,0.064780,40.161699,146.620473,212135.778438,0.101305,7.901127,0.199491,1.560479,0.982157,0.032868,0.135679,0.075228,0.078591,0.000006,0.001335,0.002589,259.793237,0.044857,77.830151,19593.231963,29.589607,44910.765796,11.378634,42.956295,2626.716424,0.133473,0.078886,0.043950,14876.816277,0.000065,0.186773,9.547539,123359.088405,1287.382772,0.091167,0.146115,366.852698,8.731347,0.001972,0.105246,13928.508294,0.001339,4.620109,2753.380027,213.063381,0.222348,0.202651,0.126415,51.826506,3.398567,3.398567,1012.672179,598.705666,692.043666,568.917635
industry,1583.568326,88.921405,20.083547,0.146900,6.197352,11.113481,0.103409,0.236301,12.254510,3.785987,10.949693,0.228776,0.571096,413.244097,0.212680,3.622719,20.573917,13.319823,8.650480,8.388080,322.509075,0.090754,83.947931,200.540867,593752.979393,0.049954,7.779679,0.186229,1.463897,0.863199,0.031929,0.100166,0.059159,0.060873,0.000004,0.001291,0.002187,1190.193633,0.034279,175.364914,65493.974894,57.434512,146472.538317,11.624001,61.748539,4797.468269,0.123596,0.083903,0.029211,23706.107200,0.000021,0.202599,9.237611,357502.153919,4488.614304,0.125196,0.189359,782.772411,10.369458,0.001613,0.077202,68929.898106,0.000266,5.521922,6738.333387,463.081839,0.226486,0.206657,0.142726,78.706806,4.928989,4.928989,1761.825822,959.152540,1002.096875,955.784127
wiehre,224.893948,35.339300,6.844542,0.105316,7.193639,6.926556,0.080437,0.200689,4.113453,1.467451,10.294301,0.232667,0.468344,200.011223,0.210364,4.050575,10.148760,7.236915,9.865434,7.266570,95.002471,0.101749,35.974371,206.184215,128067.256246,0.065223,6.656241,0.153306,1.378159,0.678514,0.053542,0.096545,0.118499,0.041495,0.000007,0.001442,0.002125,122.642231,0.053168,57.035623,11863.698750,45.441336,30695.185290,12.603002,24.318844,1039.539438,0.112588,0.069384,0.046049,5433.994475,0.000083,0.174124,10.131877,178362.126252,2631.178688,0.128865,0.151336,729.376835,11.161161,0.004759,0.102959,13117.668236,0.000897,6.771715,1380.690191,214.773197,0.208664,0.207135,0.141631,40.363755,1.761847,1.761847,259.468857,198.081927,140.592864,142.767392
centre,545.742384,60.719849,43.809090,0.129294,11.528026,8.876573,0.118198,0.227502,6.355516,2.442183,12.127138,0.265420,5.205604,344.923806,0.180097,4.009491,7.544282,4.806835,6.831684,8.183545,68.406125,0.074176,19.341756,93.306503,79241.402670,0.073596,8.300301,0.183412,1.777385,0.683540,0.026817,0.051489,0.068827,0.034866,0.00

In [29]:
from sklearn.preprocessing import StandardScaler

In [30]:
vals = StandardScaler().fit_transform(primary)
component_data = pd.DataFrame(vals, columns=primary.columns, index=primary.index)
component_data = component_data.drop(component_data.columns[component_data.std() == 0], axis=1)
vals = np.nan_to_num(component_data)
standardised_data = pd.DataFrame(vals, columns=component_data.columns, index=component_data.index)

In [31]:
standardised_data.describe().iloc[3:].style.background_gradient(axis=1, cmap="BuGn")

,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,ssbCCM,ssbCCD,stbOri,mtbSWR,libNCo,ldbPWL,ltcBuA,mtbAli,mtbNDi,ltbIBD,stbCeA,stbSAl,sdsLen,sssLin,ldsMSL,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,sddAre,midRea,midAre,stcOri,sdcLAL,sdcAre,sscCCo,sscERI,mtcWNe,mdcAre,ltcWRB,sicCAR,stcSAl,ldkAre,ldkPer,lskCCo,lskERI,lskCWA,ltkOri,ltkWNB,likWBB,sdsAre,likWCe,mibCou,mibAre,mibLen,mibElo,mibERI,mibCCo,mibLAL,mibFR,mibSCo,micBAD,licBAD,misBAD,midBAD
min,-0.330961,-0.992937,-0.016051,-4.593403,-1.199103,-0.376141,-9.982132,-3.072990,-1.218052,-0.537840,-1.584743,-1.075547,-0.154405,-0.586918,-3.080010,-0.866075,-0.935281,-1.551593,-0.742182,-0.868304,-0.582990,-6.375284,-1.325545,-1.887213,-1.090064,-1.673327,-3.371280,-2.266465,-2.864105,-2.008258,-1.763950,-6.170528,-1.271539,-1.678201,-1.889772,-0.665402,-2.003037,-0.507673,-0.514794,-0.632542,-0.482484,-1.501457,-0.703845,-1.655195,-1.135056,-0.380546,-3.182002,-9.180890,-1.889276,-0.649556,-1.293139,-1.447880,-0.976420,-0.555298,-0.604376,-2.473540,-2.339248,-0.748162,-1.701168,-0.618976,-1.840812,-0.378226,-0.007901,-0.507460,-0.322230,-0.521299,-2.570528,-3.619156,-2.915751,-0.860968,-1.406053,-1.406053,-0.386439,-0.658289,-0.505363,-0.531380
25%,-0.266007,-0.524340,-0.016051,-0.442138,-0.503167,-0.360145,-0.216002,-0.740627,-0.557010,-0.534683,-0.886126,-1.075547,-0.154405,-0.413579,-0.689289,-0.729466,-0.504225,-0.555246,-0.720141,-0.750115,-0.374974,0.133400,-0.460334,-0.735767,-0.563687,-0.681539,-0.599349,-0.693711,-0.608614,0.181427,-0.672349,-0.566425,-0.686828,-0.718878,-0.729137,-0.169901,-0.450814,-0.399860,-0.514794,-0.337879,-0.364302,-0.737962,-0.464336,-0.888033,-0.519942,-0.317441,-0.683843,-0.470797,-0.614463,-0.480400,-0.788037,-0.733116,-0.804773,-0.500446,-0.487343,-0.719406,-0.853935,-0.642455,-0.884879,-0.372493,-0.763973,-0.312347,-0.006169,-0.507460,-0.282934,-0.378706,-0.747963,-0.537437,-0.734655,-0.545459,-0.530046,-0.530046,-0.311484,-0.511287,-0.364012,-0.382952
50%,-0.167632,-0.189042,-0.016051,0.202102,-0.503167,-0.337579,0.545738,0.022884,-0.163910,-0.485588,-0.023837,-0.089855,-0.154405,-0.286247,0.092452,-0.340670,-0.235032,-0.298727,-0.475039,-0.390336,-0.243633,0.435980,-0.290322,-0.189228,-0.299987,-0.104966,-0.088519,-0.221110,0.044857,0.181427,-0.131847,0.044021,-0.245537,-0.106896,-0.028306,-0.030435,0.059748,-0.253644,-0.514794,-0.232637,-0.271903,-0.160183,-0.284450,-0.011310,-0.277998,-0.250989,0.081972,0.218411,-0.088293,-0.354418,-0.209451,-0.141645,-0.358492,-0.387245,-0.374038,-0.034639,0.315086,-0.472495,-0.039972,-0.106749,-0.054719,-0.248093,-0.004824,-0.379835,-0.236418,-0.273266,0.009175,0.336857,0.046167,-0.322991,-0.187650,-0.187650,-0.268409,-0.379073,-0.290089,-0.301923
75%,-0.025138,0.169427,-0.016051,0.738503,0.192768,-0.213404,0.560718,0.817244,0.171237,0.232800,0.857382,0.606925,-0.154405,-0.006118,0.737699,0.377456,0.132704,0.204090,0.381493,0.397701,-0.030922,0.476418,0.114821,0.558946,0.190144,0.460522,0.457979,0.476265,0.631606,0.181427,0.576775,0.695515,0.471350,0.600192,0.634647,0.110024,0.405015,-0.016782,0.349102,-0.042826,-0.085394,0.479502,0.044477,0.866063,0.076757,-0.118803,0.740522,0.677252,0.467869,-0.009176,0.560371,0.528610,0.535355,0.093292,0.096286,0.765533,0.926906,0.344715,0.810439,0.280029,0.541415,-0.112053,-0.002535,0.003039,-0.107963,-0.041357,0.831765,0.878081,0.833400,0.129911,0.253826,0.253826,-0.101907,0.080077,-0.059293,-0.049618
max,78.687404,32.831827,234.742626,4.282142,41.078980,11.581847,2.135946,1.634094,21.905252,47.797515,1.737693,3.197325,20.553572,22.672925,1.544257,8.365886,30.869367,12.774633,3.955099,4.083499,15.559827,0.477394,17.637295,4.405943,6.430633,11.035102,3.584028,2.676477,5.095678,2.371112,4.480010,2.675163,4.452474,7.435372,65.628396,145.609610,108.309144,13.935705,9.420010,16.534236,9.180909,4.771579,8.770038,1.728293,26.769523,30.520043,4.145314,2.340390,53.693179,21.066499,49.6668

In [32]:
# etcs.loc[morphotope_etcs[morphotope_etcs.types == 'houses'].index].explore(perfer_canvas=True, bassemap='cartodbpositron')

In [83]:
def morphotopes_to_etcs(region_id, etcs=True, model_params='_75_0_None_None_False'):


    if etcs:
        etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/tessellations/tessellation_{region_id}.parquet')

    else:
        etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/buildings/buildings_{region_id}.parquet')
        
    etcs['label'] = -1
    
    morphotopes = pd.read_parquet(f'/data/uscuni-ulce/processed_data/morphotopes/tessellation_labels_morphotopes_{region_id}{model_params}.pq')
    morphotopes.loc[:, 'morphotope_label'] =  morphotopes.values[:, 0]

    morph_dict = pd.Series(np.arange(np.unique(morphotopes.values).shape[0]),
                       np.unique(morphotopes.values))
    etcs.loc[morphotopes.index, 'label'] = morphotopes.map(lambda x: morph_dict.loc[x]).values
    etcs['morph'] = str(region_id) + '_' + '-1'
    etcs.loc[morphotopes.index, 'morph'] = str(region_id) + '_' + morphotopes.values
    return etcs

In [36]:
etcs = morphotopes_to_etcs(region_id)

In [37]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.SolidPolygonLayer.from_geopandas(etcs, opacity=.08)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 1.35 s, sys: 97.4 ms, total: 1.45 s
Wall time: 1.45 s


In [38]:
from sidecar import Sidecar
sc = Sidecar(title='Final Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [39]:
from core.cluster_validation import get_color
layer.get_fill_color = get_color(etcs.label)

In [40]:
etcs['barea'] = buildings.area
etcs['barea'] = etcs['barea'].fillna(0)

In [41]:
street_graph = mm.gdf_to_nx(streets, preserve_index=True)
nodes, edges = mm.nx_to_gdf(street_graph, spatial_weights=False)


In [42]:
blg_nid = mm.get_nearest_street(
    buildings, edges
)

In [43]:
# %%time
# links = mm.get_network_ratio(etcs, edges)
# etcs[['edgeID_keys', 'edgeID_values']] = links
# keys = etcs.edgeID_values.apply(lambda a: np.argmax(a))
# tess_nid = np.array([inds[i] for inds, i in zip(etcs.edgeID_keys, keys)])

# etc_nid = pd.Series(tess_nid, index=etcs.index)

In [44]:
etc_nid = mm.get_nearest_street(
    etcs, edges
)

In [45]:
%%time
inpidxs, tree_idxs = streets.sindex.query(etcs.geometry, predicate='dwithin', distance=1)

CPU times: user 645 ms, sys: 998 μs, total: 646 ms
Wall time: 652 ms


In [46]:
street_etcarea_std = etcs.iloc[inpidxs].area.groupby(streets.index[tree_idxs]).std()
street_etcarea_std = street_etcarea_std.fillna(0)
etc_street_std = etc_nid.map(lambda x: street_etcarea_std.loc[x] if x in street_etcarea_std else 0)

In [47]:
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
from libpysal.graph import read_parquet, Graph

graph = read_parquet(graph_dir + f'tessellation_graph_{region_id}.parquet')

In [48]:
# graph = Graph.build_fuzzy_contiguity(etcs.geometry, tolerance=.1) 

In [49]:
graph3 = graph.higher_order(k=3, lower_order=True, diagonal=True)

In [50]:
# differences between apartment blocks and industrial - 
# ssbEri (more rectangular),
# ssbCCD ( more complex patterns), 
# sdsLen ( simpler streets), 
# ldsMSL(longer, simpler streets), 
# ldsRea(more buildings), 
# ldsAre(more built up area), 
# ldsCDL(simpler streets), 
# mtdMDi(simpler streets), 
# sddAre(more built up area), 
# midRea(more buildings), 
# sdsAre(more built up area)



# stds
# sdbAre(less regular patterns) - std in etc neighbours, - seems to work - barea_std, baarea3_std

# sssLin(less complex streets) - std in etc neighbours, - meh

# sdsAre (less regular patterns& more built up), - meh

# sddAre (less regular patterns & more built up), - difficult

# midRea	(less regular patterns& more built up), 
# midAre (less regular patterns & more built up), 

# ldsAre (more built up) - std in large 3 neighbours, 
# ldsCDL(less regular streets) - std in large 3 neighbours, 

In [51]:
## variety stuff

In [52]:
# barea-stds - good
etcs['barea'] = buildings.area
etcs['barea'] = etcs['barea'].fillna(0)
assert (etcs.loc[buildings.index, 'barea'] == buildings.area).all()

res = graph.describe(etcs['barea'])
etcs['micBAD'] = res['std']

In [53]:
res = graph3.describe(etcs['barea'])
etcs['barea3_std'] = res['std']

In [54]:
# variability in street attached buildings area
street_barea_std = buildings.area.groupby(blg_nid).std()
street_barea_std = street_barea_std.fillna(0)
blg_street_std = blg_nid.map(lambda x: street_barea_std.loc[x])

etcs['blg_street_std'] = blg_street_std
etcs['blg_street_std'] = etcs['blg_street_std'].fillna(0)

In [55]:
etcs['blg_street_std_3'] = graph3.describe(etcs['blg_street_std'])['mean']

In [56]:
# variability in node attached buildings area

bnode_id = mm.get_nearest_node(
    buildings, nodes, edges,  blg_nid
)


node_barea_std = buildings.area.groupby(bnode_id).std()
node_barea_std = node_barea_std.fillna(0)
blg_node_std = bnode_id.map(lambda x: node_barea_std.loc[x])

etcs['blg_node_std'] = blg_node_std
etcs['blg_node_std'] = etcs['blg_node_std'].fillna(0)

etcs['blg_node_std_3'] = graph3.describe(etcs['blg_node_std'])['mean']

In [57]:
# # variability in street attached built up area - meh
etcs['etc_street_std'] = graph.describe(primary['sdsAre'])['std'].fillna(0)
etcs['etc_street3_std'] = graph3.describe(primary['sdsAre'])['std'].fillna(0)

In [58]:
# linearity in different ways - meh
etcs['sssLin_std'] = graph.describe(primary['sssLin'])['std']
etcs['sssLin3_std'] = graph3.describe(primary['sssLin'])['std']

In [59]:
# cell area variability - meh
res = graph.describe(tess['sdcAre'])
etcs['tarea_std'] = res['std']

# cell area variabilit inside enclosure - meh
enc_etcarea_std = etcs.area.groupby(etcs['enclosure_index']).std().fillna(0)
etcs['enc_etcarea_std'] = etcs['enclosure_index'].map(lambda x: enc_etcarea_std.loc[x])

In [65]:
etcs['non_building_area'] = etcs.area - etcs['barea']

In [67]:
etcs['non_building_area_knn1'] = graph.describe(etcs['non_building_area'])['mean']
etcs['non_building_area_knn1_std'] = graph.describe(etcs['non_building_area'])['std']


In [68]:
# ## doesnt really add much. mean is basically area
# etcs['cor_std'] = res['std']
# etcs['cor_mean'] = res['mean']

# etcs['cor_std'] = etcs['cor_std'].fillna(0)
# etcs['cor_mean'] = etcs['cor_mean'].fillna(0)

# res = graph.describe(etcs['cor_std'])
# etcs['cor_std'] = res['mean']

# res = graph.describe(etcs['cor_mean'])
# etcs['cor_mean'] = res['std']

In [69]:
## nothing much
etcs['sicar'] = etcs['barea'] / etcs.area
res = graph.describe(etcs['sicar'])
etcs['sicar_std'] = res['mean']

In [70]:
etcs['bperi'] = buildings.length
etcs['bperi'] = etcs['bperi'].fillna(0)
res = graph.describe(etcs['bperi'])
etcs['bperi_std'] = res['std']

In [71]:
import momepy as mm

In [72]:
etcs['fr'] = mm.facade_ratio(etcs)

In [73]:
res = graph.describe(etcs['fr'])
etcs['fr_std'] = res['std']

In [74]:
enc_etcarea_std = etcs.area.groupby(etcs['enclosure_index']).std().fillna(0)
etcs['enc_etcarea_std'] = etcs['enclosure_index'].map(lambda x: enc_etcarea_std.loc[x])

In [76]:
morphotope_etcs = etcs.loc[morphotopes.index]
morphotope_etcs['mlabel'] = morphotopes.values
morphotope_etcs['types'] = morphotope_etcs['mlabel'].str.split('_').str[-1]

In [79]:
res = morphotope_etcs.iloc[:, 4:-2].groupby(morphotope_etcs.types).mean().loc[['blocks', 'commercial', 'industry', 'wiehre', 'centre']]
res.style.background_gradient(axis=0, cmap="BuGn")

,barea,micBAD,barea3_std,blg_street_std,blg_street_std_3,blg_node_std,blg_node_std_3,etc_street_std,etc_street3_std,sssLin_std,sssLin3_std,tarea_std,enc_etcarea_std,non_building_area,non_building_area_knn1,non_building_area_knn1_std,sicar,sicar_std,bperi,bperi_std,fr,fr_std
types,,,,,,,,,,,,,,,,,,,,,,
blocks,341.742287,572.034326,686.969474,388.219860,397.811212,400.766791,411.083774,7940.050278,15925.000639,0.034228,0.074904,1741.704062,1573.721049,956.789106,1180.037821,1291.848016,0.322394,0.323849,73.552114,64.931164,6.537328,3.347853
commercial,350.136787,631.806648,799.767574,445.852679,436.521603,487.254321,488.121457,5989.780323,12282.080931,0.021885,0.047505,1846.553245,1242.001233,829.116927,1115.225583,1296.487578,0.294777,0.299636,64.751403,57.062179,6.066835,3.531241
industry,638.146841,1384.256582,1537.914378,1068.097757,1038.867598,1141.803853,1090.245612,16814.976467,33150.596825,0.026185,0.054840,3500.968956,3401.860238,1991.738508,2423.763862,2328.335329,0.251364,0.268310,89.243874,89.690067,9.326919,5.254425
wiehre,144.041218,175.966384,235.119842,150.788570,156.185084,161.008929,164.863418,3965.663759,7505.393046,0.019291,0.040320,708.764280,613.628024,433.341175,551.012077,564.970902,0.298074,0.301408,47.093384,32.161859,4.673437,2.263856
centre,283.687588,475.419516,539.220298,254.733544,267.168215,300.586341,308.676164,3766.664444,8006.687168,0.023305,0.042716,1113.960264,674.896624,371.366279,509.870869,705.412651,0.541005,0.540640,70.254950,65.009112,4.510375,2.844372


In [48]:
# barea_stds and blg_street_stds, etc_street3_std, sssLin, 

In [49]:
### fixed minimum for buildings and tess area to limit all the weird variability... 

In [134]:
import lonboard
from sidecar import Sidecar

In [135]:
plotting = etcs[etcs.index >= 0]

In [136]:
plotting.loc[:, 'barea'] = plotting.barea.clip(1, ).fillna(0)
plotting.loc[:, 'barea_std'] = plotting.barea_std.clip(1, )

In [126]:
layer = lonboard.SolidPolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [127]:
sc = Sidecar(title='tess')
m = lonboard.Map(layer)
with sc:
    display(m)

In [128]:
etcs.columns

Index(['enclosure_index', 'geometry', 'label', 'morph', 'barea', 'sssLin_std',
       'sssLin3_std', 'barea_std', 'barea3_std', 'tarea_std',
       'enc_etcarea_std', 'blg_street_std', 'etc_street_std', 'cor_std',
       'cor_mean', 'sicar', 'sicar_std', 'bperi', 'bperi_std', 'fr', 'fr_std',
       'sssLin_std3', 'blg_street3_std'],
      dtype='object')

In [156]:
column = 'blg_street3_std'

In [157]:
# barea_std, barea3_std

In [169]:
from lonboard.colormap import apply_continuous_cmap
import matplotlib as mpl
from mapclassify import classify

classifier = classify(plotting[column], 'quantiles', k=100)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/mapclassify/classifiers.py:1653: UserWarning: Not enough unique values in array to form 100 classes. Setting k to 95.
  self.bins = quantile(y, k=k)


In [159]:
from core.cluster_validation import get_color
a = np.log(plotting[column])
# quintiles = np.nanpercentile(a, np.arange(1, 100, 1).tolist() + [99.5, 99.9])    
quintiles = np.nanpercentile(a, np.arange(1, 100, 1).tolist())    
vals = np.searchsorted(quintiles, a)
layer.get_fill_color = get_color(vals)

In [63]:
locs = [445777, 445835, 445963]

In [79]:
# etcs.loc[locs].explore()

In [66]:
blg_nid.loc[locs]

445777    74418.0
445835    74417.0
445963    47646.0
dtype: float64